In [1]:
from arraytainers import Numpytainer, Jaxtainer
import numpy as np
from numpy import array
import jax
import jax.numpy as jnp
import functools
from jax.numpy import DeviceArray

In [3]:
a = Numpytainer({'mean': jnp.array([1]), 'chol_diag': jnp.array([1]), 'chol_lowerdiag': jnp.array([0])})#{'a': {'c':[np.random.rand(6,3,5,2)], 'd':np.random.rand(6,3,5,2)}, 'b': np.random.rand(3,3,5,5), 'e':2}
# A = Numpytainer(a)
np.ones(a)

mean
chol_diag
chol_lowerdiag
mean
chol_diag
chol_lowerdiag
mean
chol_diag
chol_lowerdiag
mean
chol_diag
chol_lowerdiag


Numpytainer({'mean': array([2.71828183]), 'chol_diag': array([2.71828183]), 'chol_lowerdiag': array([1.])})

In [ ]:
A.filter(['c'], 'a').shape

In [ ]:
a = {'a': np.random.rand(4,3,5,5), 'b': np.random.rand(3,3,5,5)}
A = Jaxtainer(a)
A.reshape(A.shape[:-2], 25).shape

In [ ]:
a = {'a': np.random.rand(4,3,5), 'b': np.random.rand(3,4,5,5)}
A = Jaxtainer(a)
A.reshape(12, A.shape[2:]).shape